In [4]:
import IPython 
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [9]:
def train_test() -> None:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))
    x_train, x_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)

    model = SVC(gamma='scale')
    model.fit(x_train, y_train)

    test_acc = model.score(x_test, y_test)

    print(f"Test accuracy: {test_acc}")




In [10]:
train_test()

Test accuracy: 0.9416666666666667


converting it into a data pipeline using Zenml

In [2]:
from zenml import pipeline, step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

In [5]:
@step
def importer() -> tuple[
    Annotated[np.ndarray, 'x_train'],
    Annotated[np.ndarray, 'x_test'],
    Annotated[np.ndarray, 'y_train'],
    Annotated[np.ndarray, 'y_test']
]:
    digits = load_digits()
    data = digits.images.reshape((len(digits.images), -1))

    x_train, x_test, y_train, y_test = train_test_split(data, digits.target, test_size=0.2, shuffle=False)

    return x_train, x_test, y_train, y_test


In [6]:
@step
def svc_trainer(x_train: np.ndarray, y_train: np.ndarray) -> ClassifierMixin:
    model = SVC(gamma=0.001)
    model.fit(x_train, y_train)
    return model

In [7]:
@step
def svc_tester(x_test: np.ndarray, y_test: np.ndarray, model: ClassifierMixin) -> float:
    model_score = model.score(x_test, y_test)
    print(f"model score: {model_score}")
    return model_score


Creating a Pipeline using Zenml 

In [9]:
@pipeline
def deigit_pipeline():
    x_train, x_test, y_train, y_test = importer()
    model = svc_trainer(x_train, y_train)
    eval = svc_tester(x_test, y_test, model)

Running Zenml Pipeline


In [10]:
degit_svc_pipeline = deigit_pipeline()

Initiating a new run for the pipeline: deigit_pipeline.
PyTorch version 2.7.1+cu118 available.
Registered new pipeline: deigit_pipeline.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Step importer has started.
[importer] generated new fontManager
Step importer has finished in 2.717s.
Step svc_trainer has started.
Step svc_trainer has finished in 0.251s.
Step svc_tester has started.
[svc_tester] model score: 0.9583333333333334
Step svc_tester has finished in 0.605s.
Pipeline run has finished in 6.878s.
